In [1]:
import pandas as pd

In [10]:
rMatPath = '/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/05_abundanceDistributions/matrix.counts.readsXContigs.remotion.ForAlpha.csv'
allSeqsPath = '/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/01_Derep/allContigs.DeRep.fna'

In [4]:
rMat = pd.read_csv(rMatPath,header=0,index_col=0,sep='\t') #rMat:=Recruitment Matrix
rMat.head()


,10002_K1_8Lpp_out_L611,10002_K2_8Lpp_out_L539,10002_T1_8Lpp_out_L620,10003_K1_8Lpp_out_L711,10003_T1_8Lpp_out_L570,10004_K1_8Lpp_out_L1129,10004_T1_8Lpp_out_L664,10006_K1_8Lpp_out_L555,10006_O1_8Lpp_out_L1891,10007_Da1_8Lpp_out_L1459,...,9_Da1_8Lpp_out_L1884,9_E1_8Lpp_out_L536,9_H1_8Lpp_out_L10636,9_J1_8Lpp_out_L47237,9_K2_8Lpp_out_L11520,9_M1_8Lpp_out_L809,9_O2_8Lpp_out_L848,9_P1_8Lpp_out_L5890,9_S1_8Lpp_out_L948,9_W2_8Lpp_out_L2103
Ab1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.069900,0.0,0.0,0.0,0.0,0.0,0.0
Ab2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.049014,0.017180,0.0,0.0,0.0,0.0,0.0,0.0
B1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.793995,0.0,0.000000,...,0.000000,0.0,3.252818,0.335648,0.0,0.0,0.0,0.0,0.0,0.0
B2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Da1,0.0,0.0,0.0,0.0,0.0,0.0,0.525969,0.000000,0.0,1.142169,...,2.478424,0.0,0.000000,0.394447,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
contigs = rMat.columns.values
contigs.tofile('/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/05_abundanceDistributions/contigsNames.ForAlpha.csv',
               sep='\n',format="%s")

### GetContigs

In [12]:
#as in utils.py
from Bio import SeqIO
def getSeqs(path,keysPath,outfile):
    """Return sequences in a list of desired sequences
    Parameters:
    path = paht of the fasta file. String
    keys = file with a list of identifiers. String
    outfile = outfile name. String
    Returns:
    Void
    Writes a file with the desired sequences"""
    seqs = {}
    for record in SeqIO.parse(path,"fasta"):
        seqs[record.id] = record

    keys = []
    fr = open(keysPath)
    for line in fr:
        keys.append(line.strip('\n'))
    fr.close()

    desiredSeqs = []
    for key in keys:
        desiredSeqs.append(seqs[key])

    SeqIO.write(desiredSeqs, outfile, "fasta")

In [16]:
getSeqs(allSeqsPath,
        '/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/05_abundanceDistributions/contigsNames.ForAlpha.csv',
        '/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/05_abundanceDistributions/contigs.ForAlpha.fna')

### Get proteins

In [17]:
def parseContigName(contigsList):
    parsed = []
    for c in contigsList:#c := contig
        c = c.split('_')
        parsed.append(c[1]+"_contig"+c[0])
    return parsed

In [18]:
contigsParsed = parseContigName(contigs)

In [24]:
seqs = {}
for record in SeqIO.parse("/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/00_Genes/Combined_InteMAP_pp_contigs.fa_LargerThan500.faa_USEDForDiamond","fasta"):
    seqs[record.id] = record

In [27]:
byFeatures = {}
for record in seqs:
    feature = record.split(">")[1]
    if feature in byFeatures:
        proteins = byFeatures[feature]
        proteins.append(record)
        byFeatures[feature] = proteins
    else:
        byFeatures[feature] = [record]

In [32]:
desiredSeqs = []
noProteinContigs = []
for feature in contigsParsed:
    try:
        for record in byFeatures[feature]:
            desiredSeqs.append(seqs[record])
    except:
        noProteinContigs.append(record)

In [36]:
print "%d of %d contigs do not have a proteins associated" %(len(noProteinContigs),len(contigsParsed))

1537 of 66446 contigs do not have a proteins associated


In [38]:
SeqIO.write(desiredSeqs,
            '/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/05_abundanceDistributions/Combined_InteMAP_pp_contigs.fa_LargerThan500.ForAlpha.faa',
            "fasta")

160232